In [2]:
import numpy as np
from Class_Preprocessing import *
import matplotlib.pyplot as plt
import Class_Image as Ci
import pandas
import tensorflow as tf
import math

In [12]:
df = pandas.read_csv("Z:/duels_question_3.csv", header=None)
#df = df.drop(np.arange(len(df.index) * 0.8 ,len(df.index)-1)) #drop val data

In [13]:
#https://metinmediamath.wordpress.com/2013/11/27/how-to-calculate-the-elo-rating-including-example/
images = {}
values_left = {'left' : 1, 'right' : 0, 'No preference' : 0.5}
values_right= {'left' : 0, 'right' : 1, 'No preference' : 0.5}

starting_elo = 1000.0

K = 32 #Arbitrary K factor, determines impact of matches

for e in range(50):
    avg_delta=0
    
    df = df.sample(frac=1) #Shuffle rows
    
    #Elo algorithm:
    for index, row in df.iterrows():
        if row[0] not in images:
            images[row[0]] = starting_elo
        if row[1] not in images:
            images[row[1]] = starting_elo
            
        R0 = 10**(images[row[0]]/400)
        R1 = 10**(images[row[1]]/400)

        E0 = R0/(R0 + R1)
        E1 = R1/(R0 + R1)

        S0 = values_left[row[2]]
        S1 = values_right[row[2]]

        left_delta =  K * (S0 - E0)
        right_delta = K * (S1 - E1)
        
        images[row[0]] += left_delta
        images[row[1]] += right_delta
        
        avg_delta += abs(left_delta)
        avg_delta += abs(right_delta)
    
    avg_delta = avg_delta/(2*len(df.index))
    
    #Print results every 5 iterations:
    if(e % 5 != 0):
        continue
        
    std_ties = 0
    preserves = 0
    for index, row in df.iterrows():
        if(row[2] == "No preference"):
            std_ties += (images[row[0]]  - images[row[1]]) ** 2
            continue

        if(row[2] == "right" and images[row[0]]  < images[row[1]]):
            preserves += 1
        elif(row[2] == "left" and images[row[0]]  > images[row[1]]):
            preserves += 1

    std_ties = math.sqrt(std_ties/len(df.index))
    preserves = (preserves/len(df.index)) * 100
    print("Iteration : {}, Avg delta : {:.1f}, std of ties : {:.2f}, preservation percentage : {:.2f}%"
          .format(e, avg_delta, std_ties, preserves))
        
images = {k: v for k, v in sorted(images.items(), key=lambda item: item[1])} #sort by keys

Iteration : 0, Avg delta : 10.5, std of ties : 10.54, preservation percentage : 63.89%
Iteration : 5, Avg delta : 7.3, std of ties : 36.98, preservation percentage : 65.00%
Iteration : 10, Avg delta : 5.8, std of ties : 45.16, preservation percentage : 65.30%
Iteration : 15, Avg delta : 4.9, std of ties : 47.13, preservation percentage : 65.38%
Iteration : 20, Avg delta : 4.3, std of ties : 46.75, preservation percentage : 65.49%
Iteration : 25, Avg delta : 3.8, std of ties : 45.47, preservation percentage : 65.55%
Iteration : 30, Avg delta : 3.5, std of ties : 43.80, preservation percentage : 65.59%
Iteration : 35, Avg delta : 3.2, std of ties : 42.12, preservation percentage : 65.63%
Iteration : 40, Avg delta : 2.9, std of ties : 40.45, preservation percentage : 65.66%
Iteration : 45, Avg delta : 2.7, std of ties : 38.87, preservation percentage : 65.68%


In [11]:
#save as tfrecord

keys = list(images.keys())
values = list(images.values())
img_path = "Z:/Sample_web_green"

tfrecord = tf.io.TFRecordWriter("Z:/data_train_synthetic.tfrecord")

for i,(k,v) in enumerate(images.items()):
    img_1 = k
    img_2 = None
    
    for i1 in range(i, len(values)):
        if values[i1] > v + (3*std_ties):
            img_2 = keys[i1]
            break
        
    if(img_2 is None):
        break
    #print(images[img_1])
    #print(images[img_2])
    
    img_1 = np.array(Ci.Image(get_filename_from_key(img_1, img_path)).preprocess_image(224)) #Loser
    img_2 = np.array(Ci.Image(get_filename_from_key(img_2, img_path)).preprocess_image(224)) #Winner
    
    if(bool(random.getrandbits(1))): #50/50 left or right image wins
        img_left = img_1
        img_right = img_2
        data_label = np.array([0,1])
    else:
        img_left = img_2
        img_right = img_1
        data_label = np.array([1,0])
    
    features = {
        'data_label' : tf.train.Feature(int64_list=tf.train.Int64List(value=data_label)),
        'labels_score': tf.train.Feature(int64_list=tf.train.Int64List(value=np.array([0]))), #Filler score value
        'data_left': tf.train.Feature(int64_list=tf.train.Int64List(value=img_left.astype(int).flatten())),
        'data_right': tf.train.Feature(int64_list=tf.train.Int64List(value=img_right.astype(int).flatten())),
            
        }
    
    example = tf.train.Example(features=tf.train.Features(feature=features))
    tfrecord.write(example.SerializeToString())


KeyboardInterrupt: 